In [30]:
import os
import json

# Set data root directory path
# Assuming the script runs from the repository root, data is in assets/data
ROOT_DIR = os.path.join("../assets", "data")

# Directories to check for solver/parser/verifier files
PUZZLES_DIR = "../Puzzles"
CRAWLERS_DIR = "../Crawlers"
COMMON_PARSERS_DIR = os.path.join(PUZZLES_DIR, "Common", "Parser", "PuzzleParsers")
# COMMON_VERIFIERS_DIR = os.path.join(PUZZLES_DIR, "Common", "Verifier", "PuzzleVerifiers")

def get_max_size(puzzles_dict):
    """
    Traverse puzzles dictionary, parse dimensions from first line, 
    return max size string (compared by area).
    Returns "-" if unable to get dimensions.
    """
    max_area = -1
    max_size_str = "-"

    for key, val in puzzles_dict.items():
        if not isinstance(val, dict):
            continue
            
        problem_str = val.get("problem", "")
        if not problem_str:
            continue

        # Get first line
        first_line = problem_str.strip().split('\n')[0].strip()
        tokens = first_line.split()

        # Try to read first two numbers
        if len(tokens) >= 2:
            try:
                # Assume format is "Width Height" or "Rows Cols"
                # Parse as numbers to compute area, no distinction between width/height
                dim1 = int(tokens[0])
                dim2 = int(tokens[1])
                area = dim1 * dim2
                
                if area > max_area:
                    max_area = area
                    max_size_str = f"{dim1}x{dim2}"
            except ValueError:
                continue
    
    return max_size_str

def check_solver_files(puzzle_name):
    """
    Check if all required solver files exist for a puzzle.
    Returns ✅ if all exist, ❌ otherwise.
    """
    # Check for solver file
    solver_path = os.path.join(PUZZLES_DIR, f"{puzzle_name}Solver.py")
    
    # Check for parser file
    parser_path = os.path.join(COMMON_PARSERS_DIR, f"{puzzle_name}Parser.py")
    
    # Check for verifier file
    # verifier_path = os.path.join(COMMON_VERIFIERS_DIR, f"{puzzle_name}Verifier.py")
    
    # Check if all files exist
    if (os.path.exists(solver_path) and 
        os.path.exists(parser_path)):
        # os.path.exists(verifier_path)):
        return "✅"
    return "❌"

def check_crawler_file(puzzle_name):
    """
    Check if crawler file exists for a puzzle.
    Returns ✅ if exists, ❌ otherwise.
    """
    crawler_path = os.path.join(CRAWLERS_DIR, f"{puzzle_name}Crawler.py")
    
    if os.path.exists(crawler_path):
        return "✅"
    return "❌"

def generate_markdown_table():
    if not os.path.exists(ROOT_DIR):
        print(f"Error: Directory '{ROOT_DIR}' not found.")
        return

    # Get all subdirectories and sort
    subdirs = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]
    subdirs.sort()  # Sort alphabetically

    table_data = []  # Store data for each row
    total_problems = 0
    total_solutions = 0

    # Table headers
    headers = ["No.", "Puzzle Name", "Problems", "Solutions", "Max Size", "solved?", "crawler?"]
    
    # Traverse each puzzle directory
    for idx, puzzle_name in enumerate(subdirs, 1):
        puzzle_dir = os.path.join(ROOT_DIR, puzzle_name)
        
        # Build file paths
        prob_json_path = os.path.join(puzzle_dir, "problems", f"{puzzle_name}_puzzles.json")
        sol_json_path = os.path.join(puzzle_dir, "solutions", f"{puzzle_name}_solutions.json")

        # Initialize row variables
        p_count = "-"
        s_count = "-"
        max_size = "-"
        solved_status = "❌"
        crawler_status = "❌"

        # 1. Process Problems JSON
        if os.path.exists(prob_json_path):
            try:
                with open(prob_json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # Count puzzle number (based on dictionary keys, more accurate than info count)
                    puzzles = data.get("puzzles", {})
                    count = len(puzzles)
                    p_count = count
                    total_problems += count
                    try:
                        # Calculate max size
                        max_size = get_max_size(puzzles)
                    except Exception as e:
                        # Keep "-" if parsing fails
                        pass
            except Exception as e:
                # Keep "-" if parsing fails
                pass

        # 2. Process Solutions JSON
        if os.path.exists(sol_json_path):
            try:
                with open(sol_json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # Count solution number
                    solutions = data.get("solutions", {})
                    count = len(solutions)
                    s_count = count
                    total_solutions += count
            except Exception as e:
                pass

        # 3. Check solver files status
        try:
            solved_status = check_solver_files(puzzle_name)
        except Exception as e:
            # Keep ❌ if check fails
            pass

        # 4. Check crawler file status
        try:
            crawler_status = check_crawler_file(puzzle_name)
        except Exception as e:
            # Keep ❌ if check fails
            pass

        table_data.append([str(idx), puzzle_name, str(p_count), str(s_count), 
                          max_size, solved_status, crawler_status])

    # --- Generate Markdown Output ---

    # Print table header
    print(f"| {' | '.join(headers)} |")
    print(f"| {' | '.join(['---'] * len(headers))} |")

    # Print data rows
    for row in table_data:
        print(f"| {' | '.join(row)} |")

    # Print summary row
    # No. empty, Name as Total, Counts in bold, other columns empty
    print(f"| | **Total** | **{total_problems}** | **{total_solutions}** | - | - | - |")

if __name__ == "__main__":
    generate_markdown_table()

| No. | Puzzle Name | Problems | Solutions | Max Size | solved? | crawler? |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | ABCEndView | 607 | 607 | 8x8 | ❌ | ✅ |
| 2 | Akari | 970 | 970 | 100x100 | ✅ | ✅ |
| 3 | Battleship | - | - | - | ❌ | ✅ |
| 4 | Binairo | 380 | 380 | 14x14 | ✅ | ✅ |
| 5 | Bosanowa | 38 | 38 | 11x16 | ✅ | ✅ |
| 6 | Buraitoraito | 101 | 100 | 15x15 | ✅ | ❌ |
| 7 | Burokku | 270 | 270 | 10x10 | ❌ | ✅ |
| 8 | ButterflySudoku | 77 | 77 | 12x12 | ✅ | ✅ |
| 9 | Clueless1Sudoku | 29 | 29 | 27x27 | ✅ | ✅ |
| 10 | Clueless2Sudoku | 40 | 40 | 27x27 | ✅ | ✅ |
| 11 | Corral | 419 | 419 | 25x25 | ❌ | ✅ |
| 12 | CountryRoad | 270 | 270 | 15x15 | ✅ | ✅ |
| 13 | Creek | 440 | 440 | 40x50 | ❌ | ✅ |
| 14 | CurvingRoad | 190 | 190 | 14x14 | ❌ | ✅ |
| 15 | Detour | 80 | 80 | 13x12 | ❌ | ✅ |
| 16 | DiffNeighbors | 140 | 140 | 15x15 | ❌ | ✅ |
| 17 | Dominos | 580 | 579 | 10x11 | ✅ | ✅ |
| 18 | DoubleBack | 100 | 100 | 26x26 | ✅ | ✅ |
| 19 | DoubleMinesweeper | - | - | - | ❌ | ❌ |
| 

| No. | Puzzle Name | Problems | Solutions | Max Size | solved? | crawler? |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | ABCEndView | 607 | 607 | 8x8 | ❌ | ✅ |
| 2 | Akari | 970 | 970 | 100x100 | ✅ | ✅ |
| 3 | Battleship | - | - | - | ❌ | ✅ |
| 4 | Binairo | 380 | 380 | 14x14 | ✅ | ✅ |
| 5 | Bosanowa | 38 | 38 | 11x16 | ✅ | ✅ |
| 6 | Buraitoraito | 101 | 100 | 15x15 | ✅ | ❌ |
| 7 | Burokku | 270 | 270 | 10x10 | ❌ | ✅ |
| 8 | ButterflySudoku | 77 | 77 | 12x12 | ✅ | ✅ |
| 9 | Clueless1Sudoku | 29 | 29 | 27x27 | ✅ | ✅ |
| 10 | Clueless2Sudoku | 40 | 40 | 27x27 | ✅ | ✅ |
| 11 | Corral | 419 | 419 | 25x25 | ❌ | ✅ |
| 12 | CountryRoad | 270 | 270 | 15x15 | ✅ | ✅ |
| 13 | Creek | 440 | 440 | 40x50 | ❌ | ✅ |
| 14 | CurvingRoad | 190 | 190 | 14x14 | ❌ | ✅ |
| 15 | Dominos | 580 | 579 | 10x11 | ✅ | ✅ |
| 16 | DoubleBack | 100 | 100 | 26x26 | ✅ | ✅ |
| 17 | DoubleMinesweeper | - | - | - | ❌ | ❌ |
| 18 | EntryExit | 170 | 170 | 16x16 | ✅ | ✅ |
| 19 | Eulero | 290 | 290 | 5x5 | ✅ | ✅ |
| 20 | EvenOddSudoku | 129 | 129 | 9x9 | ✅ | ✅ |
| 21 | Fillomino | 840 | 840 | 50x64 | ❌ | ✅ |
| 22 | Fivecells | - | - | - | ❌ | ❌ |
| 23 | Fobidoshi | 250 | 250 | 12x12 | ❌ | ✅ |
| 24 | Foseruzu | 310 | 310 | 30x45 | ❌ | ✅ |
| 25 | Fuzuli | 160 | 160 | 8x8 | ✅ | ✅ |
| 26 | Gappy | 429 | 427 | 18x18 | ✅ | ✅ |
| 27 | Gattai8Sudoku | 120 | 120 | 21x33 | ✅ | ✅ |
| 28 | GokigenNaname | 780 | 780 | 24x36 | ❌ | ✅ |
| 29 | GrandTour | 350 | 350 | 15x15 | ✅ | ✅ |
| 30 | Hakyuu | 480 | 480 | 30x45 | ❌ | ✅ |
| 31 | Heyawake | 787 | 787 | 31x45 | ❌ | ✅ |
| 32 | Hitori | 940 | 940 | 20x20 | ✅ | ✅ |
| 33 | JigsawSudoku | 680 | 680 | 9x9 | ✅ | ✅ |
| 34 | Juosan | 80 | 80 | 30x45 | ❌ | ✅ |
| 35 | Kakkuru | 400 | 400 | 9x9 | ❌ | ✅ |
| 36 | Kakurasu | 280 | 280 | 11x11 | ✅ | ✅ |
| 37 | Kakuro | 999 | 999 | 31x46 | ✅ | ✅ |
| 38 | KillerSudoku | 810 | 810 | 9x9 | ✅ | ✅ |
| 39 | Kuromasu | 560 | 560 | 31x45 | ❌ | ✅ |
| 40 | Kuroshuto | 210 | 210 | 14x14 | ❌ | ✅ |
| 41 | LITS | 419 | 419 | 40x57 | ❌ | ✅ |
| 42 | Linesweeper | 310 | 310 | 16x16 | ✅ | ✅ |
| 43 | Magnetic | 439 | 439 | 12x12 | ✅ | ✅ |
| 44 | Makaro | 190 | 190 | 15x15 | ❌ | ✅ |
| 45 | Masyu | 828 | 828 | 40x58 | ❌ | ✅ |
| 46 | Maze-a-pix | - | - | - | ❌ | ❌ |
| 47 | Minesweeper | 360 | 360 | 14x24 | ✅ | ✅ |
| 48 | MoonSun | 200 | 200 | 30x45 | ❌ | ✅ |
| 49 | Mosaic | 165 | 104 | 118x100 | ✅ | ✅ |
| 50 | Munraito | 360 | 360 | 12x12 | ✅ | ✅ |
| 51 | Nanbaboru | 270 | 270 | 9x9 | ❌ | ✅ |
| 52 | Nondango | 110 | 110 | 14x14 | ✅ | ✅ |
| 53 | Nonogram | 2340 | 2339 | 30x40 | ✅ | ✅ |
| 54 | Norinori | 289 | 289 | 36x54 | ✅ | ✅ |
| 55 | NumberCross | 170 | 170 | 8x8 | ❌ | ✅ |
| 56 | NumberSnake | 70 | 70 | 10x10 | ❌ | ✅ |
| 57 | OneToX | 58 | 58 | 10x10 | ✅ | ✅ |
| 58 | Patchwork | 211 | 211 | 12x12 | ❌ | ✅ |
| 59 | Pfeilzahlen | 360 | 360 | 8x8 | ✅ | ✅ |
| 60 | Pills | 164 | 163 | 10x10 | ✅ | ✅ |
| 61 | Polyiamond | - | - | - | ❌ | ❌ |
| 62 | Polyminoes | - | - | - | ❌ | ❌ |
| 63 | Renban | 150 | 150 | 9x9 | ✅ | ✅ |
| 64 | SamuraiSudoku | 272 | 272 | 21x21 | ✅ | ✅ |
| 65 | Shikaku | 500 | 500 | 31x45 | ✅ | ✅ |
| 66 | ShogunSudoku | 90 | 90 | 21x45 | ✅ | ✅ |
| 67 | Simpleloop | 70 | 70 | 17x18 | ✅ | ✅ |
| 68 | Skyscraper | 470 | 470 | 8x8 | ❌ | ✅ |
| 69 | Slitherlink | 1176 | 1153 | 60x60 | ✅ | ✅ |
| 70 | Snake | 230 | 230 | 12x12 | ❌ | ✅ |
| 71 | SoheiSudoku | 120 | 120 | 21x21 | ✅ | ✅ |
| 72 | SquareO | 120 | 80 | 15x15 | ✅ | ✅ |
| 73 | Starbattle | 307 | 307 | 15x15 | ✅ | ✅ |
| 74 | Sternenhimmel | 29 | 29 | 17x17 | ❌ | ✅ |
| 75 | Str8t | 560 | 560 | 9x9 | ✅ | ✅ |
| 76 | Sudoku | 125 | 125 | 16x16 | ✅ | ✅ |
| 77 | Suguru | 200 | 200 | 10x10 | ✅ | ✅ |
| 78 | SumoSudoku | 110 | 110 | 33x33 | ✅ | ✅ |
| 79 | Tairupeinto | 268 | 268 | 15x15 | ❌ | ✅ |
| 80 | Tatamibari | 150 | 150 | 14x14 | ❌ | ✅ |
| 81 | TennerGrid | 375 | 374 | 6x10 | ✅ | ✅ |
| 82 | Tent | 706 | 706 | 30x30 | ✅ | ✅ |
| 83 | TerraX | 80 | 80 | 17x17 | ❌ | ✅ |
| 84 | Thermometer | 250 | 250 | 10x10 | ✅ | ✅ |
| 85 | TilePaint | 109 | 109 | 16x16 | ✅ | ❌ |
| 86 | Trinairo | 60 | 60 | 12x12 | ❌ | ✅ |
| 87 | WindmillSudoku | 150 | 150 | 21x21 | ✅ | ✅ |
| 88 | Yajilin | 610 | 610 | 39x57 | ❌ | ✅ |
| 89 | YinYang | 170 | 170 | 14x14 | ❌ | ✅ |
| 90 | Yonmasu | 120 | 120 | 10x10 | ❌ | ✅ |
| 91 | monotone | - | - | - | ❌ | ❌ |
| | **Total** | **29455** | **29324** | - | - | - |

In [ ]:
# import json 

# def read_json(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)  # 解析为Python列表/字典
#     return data

# # 执行读取

# raw_data_1 = read_json("../assets/data/Minesweeper/problems/Minesweeper_puzzles.json")
# raw_data_2 = read_json("../assets/data/Minesweeper/solutions/Minesweeper_solutions.json")


In [ ]:
# for k, v in raw_data_2['solutions'].items():
#     mines = v['solution'].count("x")
#     original = raw_data_1['puzzles'][k]
#     original_pzl = original['problem'].split("\n")
#     original_line1 = original_pzl[0]
#     original_pzl[0] = f"{original_line1} {mines}"
#     # print(mines)
#     raw_data_1['puzzles'][k]['problem'] = "\n".join(original_pzl)
# print(raw_data_1)


{'puzzles': {'1_4x4': {'id': '1_4x4', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/001.a.htm', 'problem': '4 4 3\n2 - - -\n- - 1 -\n- 2 - -\n- - - 1'}, '2_4x4': {'id': '2_4x4', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/002.a.htm', 'problem': '4 4 5\n- - - -\n- 1 - 2\n3 - 2 -\n- - - -'}, '3_5x5': {'id': '3_5x5', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/003.a.htm', 'problem': '5 5 8\n- - - 2 -\n2 2 - 2 -\n- - - - -\n- 2 - 2 2\n- 2 - - -'}, '4_5x5': {'id': '4_5x5', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/004.a.htm', 'problem': '5 5 7\n1 - - - 2\n- - 5 - -\n3 - - - 2\n- - 1 - -\n1 - - - 1'}, '5_5x5': {'id': '5_5x5', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/005.a.htm', 'problem': '5 5 9\n- - - - -\n1 - 4 - 4\n- - - - -\n2 - 5 - 1\n- - - - -'}, '6_5x5': {'id': '6_5x5', 'difficult': 0, 'source': 'https://www.janko.at/Raetsel/Minesweeper/006.a.htm', 'problem': '5 5

In [ ]:
# with open("../assets/data/Minesweeper/problems/Minesweeper_puzzles_2.json", 'w', encoding='utf-8') as f:
#     json.dump(raw_data_1, f, indent=2, ensure_ascii=False)

In [ ]:
# raw_data_1

{'puzzles': {'1_4x4': {'id': '1_4x4',
   'difficult': 0,
   'source': 'https://www.janko.at/Raetsel/Minesweeper/001.a.htm',
   'problem': '4 4 3\n2 - - -\n- - 1 -\n- 2 - -\n- - - 1'},
  '2_4x4': {'id': '2_4x4',
   'difficult': 0,
   'source': 'https://www.janko.at/Raetsel/Minesweeper/002.a.htm',
   'problem': '4 4 5\n- - - -\n- 1 - 2\n3 - 2 -\n- - - -'},
  '3_5x5': {'id': '3_5x5',
   'difficult': 0,
   'source': 'https://www.janko.at/Raetsel/Minesweeper/003.a.htm',
   'problem': '5 5 8\n- - - 2 -\n2 2 - 2 -\n- - - - -\n- 2 - 2 2\n- 2 - - -'},
  '4_5x5': {'id': '4_5x5',
   'difficult': 0,
   'source': 'https://www.janko.at/Raetsel/Minesweeper/004.a.htm',
   'problem': '5 5 7\n1 - - - 2\n- - 5 - -\n3 - - - 2\n- - 1 - -\n1 - - - 1'},
  '5_5x5': {'id': '5_5x5',
   'difficult': 0,
   'source': 'https://www.janko.at/Raetsel/Minesweeper/005.a.htm',
   'problem': '5 5 9\n- - - - -\n1 - 4 - 4\n- - - - -\n2 - 5 - 1\n- - - - -'},
  '6_5x5': {'id': '6_5x5',
   'difficult': 0,
   'source': 'https:/